In [1]:
from IPython.display import display

In [15]:
from dsmlibrary.datanode import DataNode
from tqdm.auto import tqdm
import requests
import pandas as pd
import dask.dataframe as dd

In [3]:
datanode = DataNode()

Init DataNode sucessful!


In [7]:
landing_dir_id = 16

In [8]:
listedCompanies_th = pd.read_csv('listedCompanies_th.csv')
companies = list(listedCompanies_th['หลักทรัพย์'])

#### Board Data

In [11]:
data_board = []
_err = []
for company in tqdm(companies):
    r = requests.get(f"https://www.set.or.th/api/set/company/{company.lower()}/board-of-director?lang=th")
    if r.status_code == 200 and r.json() != []:
        try:
            df = pd.DataFrame(r.json())
            df['positions'] = df['positions'].apply(lambda x: ",".join(x))
            df['company'] = company.lower()
        except Exception as e:
            print(e)
            _err.append(r.json())
        else:
            data_board += [df]

  0%|          | 0/827 [00:00<?, ?it/s]

In [12]:
df_data_board = pd.concat(data_board)
df_data_board

,name,positions,company
0,นาย ครรชิต สิงห์สุวรรณ์,"ประธานกรรมการ,กรรมการอิสระ,กรรมการตรวจสอบ",2s
1,นาย สมบัติ ลีสวัสดิ์ตระกูล,"ประธานกรรมการบริหาร,กรรมการ",2s
2,นาย สุรพล สติมานนท์,"รองประธานกรรมการ,กรรมการอิสระ,ประธานกรรมการตรว...",2s
3,นาย แสงรุ้ง นิติภาวะชน,"กรรมการผู้จัดการ,กรรมการ",2s
4,น.ส. บุญศรี ปัญญาเปี่ยมศักดิ์,กรรมการ,2s
...,...,...,...
1,นาย ศุภกิจ งามจิตรเจริญ,"ประธานกรรมการบริหาร,ประธานเจ้าหน้าที่บริหาร,กร...",ziga
2,น.ส. วราลักษณ์ งามจิตรเจริญ,"กรรมการผู้จัดการ,กรรมการ",ziga
3,นาง มณฑา ทัสฐาน,กรรมการ,ziga
4,นาย พลรชฏ เปียถนอม,"กรรมการอิสระ,กรรมการตรวจสอบ",ziga


In [13]:
df_data_board.to_csv('dsm.tmp/set-board.csv')

In [14]:
datanode.upload_file(directory_id=landing_dir_id, file_path='dsm.tmp/set-board.csv')

File set-board.csv alrady exists, do you want to replace y/n :  y


  0%|          | 0.00/897k [00:00<?, ?B/s]

{'id': 49,
 'msg': 'file set-board.csv create sucessful',
 'path': 'SothanaV/set-scrapping/landing/set-board.csv',
 'uploaded': True}

In [16]:
ddf_data_board = dd.from_pandas(df_data_board, npartitions=1)
ddf_data_board

,name,positions,company
npartitions=1,,,
0,object,object,object
19,...,...,...


In [18]:
datanode.write(df=ddf_data_board, directory=landing_dir_id, name="set-board", profiling=True)

{'sucess': True,
 'file_id': 61,
 'path': 'SothanaV/set-scrapping/landing/set-board.parquet'}

### Shareholder

In [20]:
all_shareholder_list = []
for company in tqdm(companies):
    r = requests.get(f"https://www.set.or.th/api/set/stock/{company.lower()}/shareholder?lang=th")    
    if r.status_code == 200 and r.json() != []:
        try:
            data = r.json()
            df = pd.DataFrame(data['majorShareholders'])
            df['company'] = company.lower()
        except Exception as e:
            print(e)
            _err.append(r.json())
        else:
            all_shareholder_list += [df]

  0%|          | 0/827 [00:00<?, ?it/s]

In [21]:
df_all_shareholder = pd.concat(all_shareholder_list)
df_all_shareholder.head()

,sequence,name,nationality,numberOfShare,percentOfShare,isThaiNVDR,company
0,1,นาย สมบัติ ลีสวัสดิ์ตระกูล,,95234050,19.05,False,2s
1,2,นาย แสงรุ้ง นิติภาวะชน,,29109087,5.82,False,2s
2,3,นาย ปรีดา วงศ์สถิตย์พร,,22397165,4.48,False,2s
3,4,น.ส. ธนิกา ตั้งพูนผลวิวัฒน์,,20048000,4.01,False,2s
4,5,น.ส. นิสาภ์ รุ่งรัตนาอุบล,,17962500,3.59,False,2s


In [22]:
df_all_shareholder.to_csv('dsm.tmp/shareholder.csv')

In [23]:
datanode.upload_file(directory_id=landing_dir_id, file_path='dsm.tmp/shareholder.csv')

File shareholder.csv alrady exists, do you want to replace y/n :  y


  0%|          | 0.00/781k [00:00<?, ?B/s]

{'id': 51,
 'msg': 'file shareholder.csv create sucessful',
 'path': 'SothanaV/set-scrapping/landing/shareholder.csv',
 'uploaded': True}

In [24]:
ddf_all_shareholder = dd.from_pandas(df_all_shareholder, npartitions=1)
ddf_all_shareholder

,sequence,name,nationality,numberOfShare,percentOfShare,isThaiNVDR,company
npartitions=1,,,,,,,
0,int64,object,object,int64,float64,bool,object
26,...,...,...,...,...,...,...


In [25]:
datanode.write(df=ddf_all_shareholder, directory=landing_dir_id, name="set-shareholder", profiling=True)

{'sucess': True,
 'file_id': 62,
 'path': 'SothanaV/set-scrapping/landing/set-shareholder.parquet'}